In [31]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=051461c27ee777e2dd2acd58ad8f1431b51d94a9d8d070f1a829b48ef15e8310
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [32]:
!pip install pyECLAT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
!pip install pyfpgrowth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.2 MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5504 sha256=26d66b96f226f20eeb0d1affb2f011bb8ef9601817ad23affc3f200274ca23f7
  Stored in directory: /root/.cache/pip/wheels/73/97/4b/f12ac994f6bbb99597396255435824c73ad3916be1e678be55
Successfully built pyfpgrowth


In [21]:
# Import headers
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
#Data preprocessing
dataset = pd.read_csv('/content/Market_Basket_Optimisation.csv', header = 0)
#Data=dataset.drop("Item(s)", axis=1)
Data=dataset
rows = len(Data.axes[0])
cols = len(Data.axes[1])
Data.columns = range(Data.shape[1])
rows

7500

In [23]:
# Intializing the list
transacts = []
# populating a list of transactions
for i in range(0, rows): 
  transacts.append([str(Data.values[i,j]) for j in range(0, cols)])


In [62]:
#Apriori algorithm
from apyori import apriori
ap = apriori(transactions = transacts, min_support = 0.2, use_colnames=True) # generate rules
# list of results coming from the apriori model
rslt = list(ap)

rslt

[RelationRecord(items=frozenset({'mineral water'}), support=0.23826666666666665, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'mineral water'}), confidence=0.23826666666666665, lift=1.0)]),
 RelationRecord(items=frozenset({'nan'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0)]),
 RelationRecord(items=frozenset({'nan', 'mineral water'}), support=0.23826666666666665, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan', 'mineral water'}), confidence=0.23826666666666665, lift=1.0), OrderedStatistic(items_base=frozenset({'mineral water'}), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({'nan'}), items_add=frozenset({'mineral water'}), confidence=0.23826666666666665, lift=1.0)])]

In [63]:
%timeit -n 100 -r 10 ap

100 loops, best of 10: 49.6 ns per loop


In [37]:
#ECLAT algorithm
from pyECLAT import ECLAT

# loading transactions DataFrame to ECLAT class
eclat = ECLAT(data=Data)

# DataFrame of binary values
eclat.df_bin


,NaN,burgers,cake,shallot,muffins,antioxydant juice,parmesan cheese,flax seed,soup,chocolate bread,...,ground beef,cauliflower,extra dark chocolate,carrots,rice,clothes accessories,whole weat flour,chicken,melons,mineral water
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7496,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
items_total = eclat.df_bin.astype(int).sum(axis=0)

items_total

NaN                       0
burgers                 654
cake                    608
shallot                  58
muffins                 181
                       ... 
clothes accessories      63
whole weat flour         69
chicken                 450
melons                   90
mineral water          1787
Length: 121, dtype: int64

In [39]:
# count items in each row
items_per_transaction = eclat.df_bin.astype(int).sum(axis=1)

items_per_transaction

0       3
1       1
2       2
3       5
4       1
       ..
7495    3
7496    6
7497    1
7498    2
7499    4
Length: 7500, dtype: int64

In [40]:
df = pd.DataFrame({'items': items_total.index, 'transactions': items_total.values}) 

# cloning pandas DataFrame for visualization purpose  
df_table = df.sort_values("transactions", ascending=False)

#  Top 5 most popular products/items
df_table.head(5)

,items,transactions
120,mineral water,1787
105,eggs,1348
69,spaghetti,1306
80,french fries,1282
33,chocolate,1229


In [55]:
eclat_indices, eclat_supports = eclat.fit(min_support=0.2)


Combination 1 by 1


1it [00:00, 50.22it/s]


Combination 2 by 2


0it [00:00, ?it/s]


Combination 3 by 3


0it [00:00, ?it/s]


In [56]:
print(eclat_supports)

{'mineral water': 0.23826666666666665}


In [57]:
%timeit -n 100 -r 10 eclat_supports

100 loops, best of 10: 36.3 ns per loop


In [44]:
import pyfpgrowth

In [64]:
patterns = pyfpgrowth.find_frequent_patterns(transacts, 20)
patterns

{('tea',): 29,
 ('nan', 'tea'): 365,
 ('chutney',): 31,
 ('chutney', 'nan'): 418,
 ('mashed potato',): 31,
 ('mashed potato', 'nan'): 424,
 ('chocolate bread',): 32,
 ('chocolate bread', 'nan'): 438,
 ('dessert wine',): 33,
 ('dessert wine', 'nan'): 458,
 ('ketchup',): 33,
 ('ketchup', 'nan'): 396,
 ('oatmeal',): 33,
 ('nan', 'oatmeal'): 426,
 ('sandwich',): 34,
 ('nan', 'sandwich'): 499,
 ('babies food',): 34,
 ('babies food', 'nan'): 494,
 ('asparagus',): 35,
 ('asparagus', 'nan'): 481,
 ('corn',): 36,
 ('corn', 'nan'): 480,
 ('salad',): 36,
 ('nan', 'salad'): 525,
 ('cauliflower',): 36,
 ('cauliflower', 'nan'): 458,
 ('shampoo',): 37,
 ('nan', 'shampoo'): 470,
 ('hand protein bar',): 39,
 ('hand protein bar', 'nan'): 508,
 ('mint green tea',): 42,
 ('mint green tea', 'nan'): 595,
 ('burger sauce',): 44,
 ('burger sauce', 'nan'): 530,
 ('pickles',): 45,
 ('nan', 'pickles'): 666,
 ('mayonnaise', 'mineral water'): 22,
 ('mayonnaise', 'mineral water', 'nan'): 253,
 ('mayonnaise', 'nan')

In [65]:
%timeit -n 100 -r 10 patterns

100 loops, best of 10: 32.5 ns per loop
